In [1]:
import os

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1 , shape = shape )
    return tf.Variable(initial)

def conv2d (x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    variable_summaries(W)
    b = bias_variable([shape[3]])
    variable_summaries(b)
    return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
    in_size = int (input.get_shape()[1])
    W = weight_variable([in_size, size])
    variable_summaries(W)
    b = bias_variable([size])
    variable_summaries(b)
    return tf.matmul(input, W) + b

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# This helper function , taken from the official TensorFlow documentation ,
# simply adds some ops that take care of logging summaries

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

LOG_DIR = 'logs/summaries'
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None , 10])
keep_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(x, [-1, 28, 28, 1])

with tf.name_scope('conv_1'):
    conv1 = conv_layer(x_image, shape = [5, 5, 1, 32])
    conv1_pool = max_pool_2x2(conv1)

with tf.name_scope('conv_2'):
    conv2 = conv_layer(conv1_pool, shape = [5, 5, 32, 64])
    conv2_pool = max_pool_2x2(conv2)
    conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
    
with tf.name_scope('full_1'):
    full_1 = tf.nn.relu(full_layer(conv2_flat , 1024))
    
with tf.name_scope('dropout'):
    full1_drop = tf.nn.dropout(full_1, keep_prob = keep_prob)
    
with tf.name_scope('activations'):
    y_conv = full_layer(full1_drop, 10)
    variable_summaries(y_conv)
    mnist = input_data.read_data_sets('tmp\data', one_hot = True)
    
with tf.name_scope('cross'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_conv, labels = y_))
    tf.summary.scalar('cross_entropy', cross_entropy)

# # SGD
# AdaGrad
train_step = tf.train.AdagradOptimizer(learning_rate = 1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries
merged = tf.summary.merge_all()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp\data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp\data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tmp\data\t10k-images-idx3-ubyte.gz
Extracting tmp\data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
saver = tf.train.Saver(max_to_keep = 7, keep_checkpoint_every_n_hours = 1)
DIR = 'models'

with tf.Session() as sess:
    
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', graph = tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test', graph = tf.get_default_graph())

    sess.run(tf.global_variables_initializer())
    start_time = time.time()
    
    for i in range (5000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            summary, train_accuracy = sess.run([merged, accuracy],
            feed_dict = {x: batch[0], y_:batch[1], keep_prob : 1.0})
            print ("time {}, step {}, training accuracy {}".format(time.time () - start_time, i, train_accuracy))
            # Add to summaries
            train_writer.add_summary(summary, i)
        if i%1000 == 0:
            saver.save(sess, os.path.join(DIR, 'model_ckpt'), global_step = i)
        sess.run(train_step, feed_dict = {x:batch[0], y_:batch[1], keep_prob : 0.5})

    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10 , 1000 , 10)
    summary , test_accuracy = sess.run([merged , accuracy], feed_dict = {x: X[1], y_: Y[1],
    keep_prob: 1.0}) 
    
    test_writer.add_summary(summary, i)
    print ("test accuracy: {}".format(test_accuracy))

time 0.5693137645721436, step 0, training accuracy 0.1599999964237213
time 21.375672340393066, step 100, training accuracy 0.30000001192092896
time 43.35503268241882, step 200, training accuracy 0.2800000011920929
time 66.34480905532837, step 300, training accuracy 0.23999999463558197
time 87.40769600868225, step 400, training accuracy 0.4399999976158142
time 109.66989302635193, step 500, training accuracy 0.4000000059604645
time 132.51177525520325, step 600, training accuracy 0.6600000262260437
time 152.6121952533722, step 700, training accuracy 0.5600000023841858
time 173.50917768478394, step 800, training accuracy 0.47999998927116394
time 193.3457715511322, step 900, training accuracy 0.6800000071525574
time 214.18678426742554, step 1000, training accuracy 0.800000011920929
time 234.2851254940033, step 1100, training accuracy 0.6600000262260437
time 254.05074000358582, step 1200, training accuracy 0.5600000023841858
time 273.7664008140564, step 1300, training accuracy 0.639999985694

### Humanity restored

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, os.path.join(DIR, 'model_ckpt-4000'))
    ans = sess.run(accuracy, feed_dict = {x: X[0], y_: Y[0], keep_prob: 1.0}) 
    print("Accuracy: {:.4}%".format(ans*100))